In [1]:
import dask.dataframe as dd
from py2neo import Graph
from tqdm.auto import tqdm
import time
from searchapp.models import Entity
from tqdm.dask import TqdmCallback

In [2]:
ddf = dd.read_csv('data/conceptnet-assertions-5.7.0.csv', header=None, sep='\t', names=['uri','relation','node_start','node_end','info'])
ddf = ddf[ddf['node_start'].str.startswith('/c/en') & ddf['node_end'].str.startswith('/c/en')]
ddf

,uri,relation,node_start,node_end,info
npartitions=158,,,,,
,object,object,object,object,object
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [3]:
def create_concepnet(df, col):
    if df.shape[0] < 3:
        return None
    names = list(df[col].unique())
    objs = [elm.name for elm in Entity.objects.filter(name__in=names)]
    data = [Entity(name=elm, attribute={}) for elm in list(set(names) - set(objs))]
    Entity.objects.bulk_create(data)

In [4]:
cb = TqdmCallback(desc="global")
cb.register()
ddf.map_partitions(lambda x: create_concepnet(x, 'node_start')).compute()

global:   0%|          | 0/158 [00:00<?, ?it/s]

0      None
1      None
2      None
3      None
4      None
       ... 
153    None
154    None
155    None
156    None
157    None
Length: 158, dtype: object